In [1]:
import sys
sys.path.insert(1, 'C:/Users/mridu/Documents/Trading Stuff/derivative')
import backtest
sys.path.insert(1, 'C:/Users/mridu/Documents/Trading Stuff/support')
from dates import *

import numpy as np
import random

here


In [2]:
# a list of dates from start to end, excluding weekends
start_date = "2023-08-06"
end_date = "2023-10-10"
first_expiry = "2023-08-10"
dates = get_weekday_dates(start_date, end_date)
print(dates)
expiries = get_expiry_dates(dates, first_expiry)
print(expiries)

here
['2023-08-07', '2023-08-08', '2023-08-09', '2023-08-10', '2023-08-11', '2023-08-14', '2023-08-15', '2023-08-16', '2023-08-17', '2023-08-18', '2023-08-21', '2023-08-22', '2023-08-23', '2023-08-24', '2023-08-25', '2023-08-28', '2023-08-29', '2023-08-30', '2023-08-31', '2023-09-01', '2023-09-04', '2023-09-05', '2023-09-06', '2023-09-07', '2023-09-08', '2023-09-11', '2023-09-12', '2023-09-13', '2023-09-14', '2023-09-15', '2023-09-18', '2023-09-19', '2023-09-20', '2023-09-21', '2023-09-22', '2023-09-25', '2023-09-26', '2023-09-27', '2023-09-28', '2023-09-29', '2023-10-02', '2023-10-03', '2023-10-04', '2023-10-05', '2023-10-06', '2023-10-09', '2023-10-10']
['2023-08-10', '2023-08-10', '2023-08-10', '2023-08-10', '2023-08-17', '2023-08-17', '2023-08-17', '2023-08-17', '2023-08-17', '2023-08-24', '2023-08-24', '2023-08-24', '2023-08-24', '2023-08-24', '2023-08-31', '2023-08-31', '2023-08-31', '2023-08-31', '2023-08-31', '2023-09-06', '2023-09-06', '2023-09-06', '2023-09-06', '2023-09-13',

In [3]:
stock = "CNXBAN"
lot_size = 15
# expiry = "2023-09-13"
# date = "2023-09-13"

averaging_method = "bilateral"

In [4]:
import os
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
# Set the current working directory to the parent directory
os.chdir(parent_directory)
os.getcwd()

'c:\\Users\\mridu\\Documents\\Trading Stuff'

In [5]:
def add_arrays(ar1, ar2):
    # convert ar1 and ar2 to lists
    ar1 = list(ar1)
    ar2 = list(ar2)
    # add the elements of the two lists
    ar3 = [ar1[i] + ar2[i] for i in range(len(ar1))]
    #convert ar3 to np array
    ar3 = np.array(ar3)
    return ar3

In [6]:
# Define the function you want to maximize
def objective_function(expiry, date, x):
    # Replace this with your actual objective function
    buy_trigs = [x[0]]
    exit_trigs = [x[1]]
    derivatives_lookback_period = int(x[2])
    derivatives_interval = int(x[3])
    averaging_lookback = int(x[4])
    averaging_sigma_1 = x[5]
    averaging_sigma_2 = x[6]

    params = [stock, expiry, date, "1second", averaging_method, [buy_trigs, exit_trigs, derivatives_lookback_period, derivatives_interval, averaging_lookback, averaging_sigma_1, averaging_sigma_2]]

    print(params)

    b1 = backtest.backtest(params[0], params[1], params[2], params[3], params[4], params[5])
    b1.backtest()
    returns = b1.get_results()
    result = (returns[0][0][0] + returns[1][0][0])*lot_size - (returns[0][0][1] + returns[1][0][1])*45 + (returns[0][0][2]*returns[0][0][3] + returns[1][0][2]*returns[1][0][3])*lot_size
    print("result of objective function = ", result)

    del b1

    return result

def set_constraints(num_variables, variables, variable_bounds):
    returns = np.zeros(num_variables)
    for i in range(num_variables):
        min_bound, max_bound = variable_bounds[i]
        returns[i] = max(min(variables[i], max_bound), min_bound)
    return returns

In [7]:
variable_bounds = [(-65, 65), (-65, 65), (1, 3), (2, 10), (1, 120), (0, 100), (0, 100)]  # Example bounds

# Define bounds for each variable (0-based index)
integer_variable_indices = [2, 3, 4]  # Indices of variables that should be integers
# Initialize variables
num_variables = 7
# variables = np.zeros(num_variables)  # Initialize variables with zeros (you can use random initialization as before)
variables = np.array([20.0, -10.0, 2, 3, 5, 2.0, 2.0])

# perturbations for calculating the derivative
deltas_list = [1, 1, 1, 1, 1, 1, 1]

# Gradient descent optimization
# learning_rate = 0.01
max_iterations = 5
tolerance = 10

# Define learning rates for each variable
# learning_rates = [0.0001, 0.0001, 0.1, 0.1, 0.1, 0.01, 0.01]  # Example learning rates

In [8]:
class gradient_descent:
    def __init__(self, stock, lot_size, expiry, date, averaging, initial_variables) -> None:
        self.stock = stock
        self.lot_size = lot_size
        self.expiry = expiry
        self.date = date
        self.averaging_method = averaging
        # self.variables = np.array([20.0, -10.0, 2, 3, 15, 5.0, 5.0])
        self.variables = initial_variables
        self.max_tries = [3, 3, 0, 2, 0, 0, 0]

    def main(self):

        global learning_rates

        for iteration in range(max_iterations):
            # Compute the gradient of the objective function
            print(iteration)
            gradient = np.zeros(num_variables)
            for i in range(num_variables):
                tries = 0
                delta = np.zeros(num_variables)
                delta[i] = deltas_list[i]  # Small perturbation for numerical gradient calculation
                while(True):
                    
                    tmp1 = objective_function(self.expiry, self.date, set_constraints(num_variables, self.variables + delta, variable_bounds))
                    tmp2 = objective_function(self.expiry, self.date, set_constraints(num_variables, self.variables - delta, variable_bounds))

                    gradient[i] = (tmp1 - tmp2) / (2 * delta[i])

                    if(tmp1 > tmp2):
                        for i in range(num_variables):
                            self.variables[i] += delta[i]
                        self.variables = set_constraints(num_variables, self.variables, variable_bounds)
                        break
                    elif(tmp1 < tmp2):
                        for i in range(num_variables):
                            self.variables[i] -= delta[i]
                        self.variables = set_constraints(num_variables, self.variables, variable_bounds)
                        break
                    else:
                        if(tries>=self.max_tries[i]):
                            self.variables[i] += delta[i] + self.max_tries[i]
                            break
                        delta[i] += 1                   

    def print_optimised(self):
        # Print the final result
        print("Optimal variables:", self.variables)
        print("Optimal value:", objective_function(self.expiry, self.date, self.variables))

    def get_optimised(self):
        return [self.variables, objective_function(self.expiry, self.date, self.variables)]

In [9]:
# an algorithm for stochastic gradient descent
# choose a date randomly, and run the normal gradient descent algorithm on that date
# take the optimised values as the initial values for another randomly chosen date

'''
class gradient_descent:
    def __init__(self, stock, lot_size, expiry, date, averaging, initial_variables) -> None:
        self.stock = stock
        self.lot_size = lot_size
        self.expiry = expiry
        self.date = date
        self.averaging_method = averaging
        # self.variables = np.array([20.0, -10.0, 2, 3, 15, 5.0, 5.0])
        self.variables = initial_variables
        self.learning_rates = [0.005, 0.005, 0.01, 0.01, 0.01, 0.01, 0.01]  # Example learning rates
        self.max_tries = [3, 3, 1, 2, 0, 0, 0]

    def main(self):

        global learning_rates
        self.learning_rates = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]  # Example learning rates

        for iteration in range(max_iterations):
            # Compute the gradient of the objective function
            print(iteration)
            gradient = np.zeros(num_variables)
            for i in range(num_variables):
                tries = 0
                delta = np.zeros(num_variables)
                delta[i] = deltas_list[i]  # Small perturbation for numerical gradient calculation
                while(True):
                    
                    tmp1 = objective_function(self.expiry, self.date, set_constraints(num_variables, self.variables + delta, variable_bounds))
                    tmp2 = objective_function(self.expiry, self.date, set_constraints(num_variables, self.variables - delta, variable_bounds))

                    gradient[i] = (tmp1 - tmp2) / (2 * delta[i])
                    # gradient[i] = (objective_function(self.expiry, self.date, set_constraints(num_variables, self.variables + delta, variable_bounds)) - objective_function(self.expiry, self.date, set_constraints(num_variables, self.variables - delta, variable_bounds))) / (2 * delta[i])
                    print("gradient = ", gradient[i])
                    if i in integer_variable_indices:
                        self.variables[i] = int(round(self.variables[i] + self.learning_rates[i] * gradient[i]))  # Ensure integer variables stay integers
                    else:
                        self.variables[i] += self.learning_rates[i] * gradient[i]
                    min_bound, max_bound = variable_bounds[i]
                    self.variables[i] = max(min(self.variables[i], max_bound), min_bound)

                    if(gradient[i]!=0):
                        break
                    else:
                        if(tries>=self.max_tries[i]):
                            break
                        tries += 1
                        if(tmp1!=0 and tmp2!=0):
                            self.learning_rates[i] = self.learning_rates[i]*1.4
                            delta[i] += 1
                        else:
                            self.learning_rates[i] = self.learning_rates[i]*0.01
                            # self.delta[i] += 1
                            

            self.learning_rates = [learning_rate*0.95 for learning_rate in self.learning_rates]

            # Check for convergence
            if np.linalg.norm(self.learning_rates * gradient) < tolerance:
                break

    def print_optimised(self):
        # Print the final result
        print("Optimal variables:", self.variables)
        print("Optimal value:", objective_function(self.expiry, self.date, self.variables))

    def get_optimised(self):
        return [self.variables, objective_function(self.expiry, self.date, self.variables)]
'''

'\nclass gradient_descent:\n    def __init__(self, stock, lot_size, expiry, date, averaging, initial_variables) -> None:\n        self.stock = stock\n        self.lot_size = lot_size\n        self.expiry = expiry\n        self.date = date\n        self.averaging_method = averaging\n        # self.variables = np.array([20.0, -10.0, 2, 3, 15, 5.0, 5.0])\n        self.variables = initial_variables\n        self.learning_rates = [0.005, 0.005, 0.01, 0.01, 0.01, 0.01, 0.01]  # Example learning rates\n        self.max_tries = [3, 3, 1, 2, 0, 0, 0]\n\n    def main(self):\n\n        global learning_rates\n        self.learning_rates = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]  # Example learning rates\n\n        for iteration in range(max_iterations):\n            # Compute the gradient of the objective function\n            print(iteration)\n            gradient = np.zeros(num_variables)\n            for i in range(num_variables):\n                tries = 0\n                delta = np.zeros(

In [10]:
no_of_stochastic_iterations = len(dates)//6
optimised_results = []
done = []

for iteration in range(no_of_stochastic_iterations):
    # pick a random date
    
    randint = random.randint(0, len(dates)-1)
    while(randint in done):
        randint = random.randint(0, len(dates)-1)
    done.append(randint)

    try:
        if(len(optimised_results)!=0):
            desc = gradient_descent(stock, lot_size, expiries[randint], dates[randint], averaging_method, optimised_results[-1][2])
        else:
            desc = gradient_descent(stock, lot_size, expiries[randint], dates[randint], averaging_method, variables)
        desc.main()
        opt_results = desc.get_optimised()
        optimised_results.append([dates[randint], expiries[randint], opt_results[0], opt_results[1]])
    except FileNotFoundError:
        no_of_stochastic_iterations += 1
        continue
    except IndexError:
        no_of_stochastic_iterations += 1
        continue
    

print(optimised_results[-1])


0
['CNXBAN', '2023-09-20', '2023-09-18', '1second', 'bilateral', [[21.0], [-10.0], 2, 3, 5, 2.0, 2.0]]
result of objective function =  -4459.885294933114
['CNXBAN', '2023-09-20', '2023-09-18', '1second', 'bilateral', [[19.0], [-10.0], 2, 3, 5, 2.0, 2.0]]
result of objective function =  -5241.552819403871
['CNXBAN', '2023-09-20', '2023-09-18', '1second', 'bilateral', [[21.0], [-9.0], 2, 3, 5, 2.0, 2.0]]
result of objective function =  -4746.548669974243
['CNXBAN', '2023-09-20', '2023-09-18', '1second', 'bilateral', [[21.0], [-11.0], 2, 3, 5, 2.0, 2.0]]
result of objective function =  -2976.8882535220937
['CNXBAN', '2023-09-20', '2023-09-18', '1second', 'bilateral', [[21.0], [-11.0], 3, 3, 5, 2.0, 2.0]]
result of objective function =  -668.4804787458487
['CNXBAN', '2023-09-20', '2023-09-18', '1second', 'bilateral', [[21.0], [-11.0], 1, 3, 5, 2.0, 2.0]]
result of objective function =  -11686.99556337027
['CNXBAN', '2023-09-20', '2023-09-18', '1second', 'bilateral', [[21.0], [-11.0], 3, 4,

In [11]:
print(*optimised_results, sep='\n')
print(done)

['2023-09-18', '2023-09-20', array([ 25., -10.,   4.,   4.,   8.,   5.,   5.]), -429.9390327713992]
['2023-09-06', '2023-09-06', array([33., -4.,  2.,  6., 11.,  8.,  8.]), -713.6237153763645]
['2023-09-08', '2023-09-13', array([31., -4.,  4., 10., 12.,  9.,  9.]), 1809.2813759344685]
['2023-08-09', '2023-08-10', array([27., -6.,  2.,  6.,  9.,  6.,  6.]), -1461.9270074865267]
['2023-10-03', '2023-10-04', array([30.,  1.,  2.,  7., 13., 10., 10.]), -1122.7841857056253]
['2023-08-18', '2023-08-24', array([31.,  6.,  2.,  6., 15., 12., 12.]), -2297.2706647084674]
['2023-10-06', '2023-10-11', array([33.,  3.,  3.,  7., 15., 12., 12.]), -4006.6974154672203]
[30, 22, 24, 2, 41, 9, 44]


In [12]:
print(len(dates))

47
